In [ ]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import random
import copy

In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Preprocess

In [ ]:
directory = './csvFiles'

In [ ]:
csv_files = [file for file in os.listdir(directory) if file.endswith('.csv')]

In [ ]:
# 파일 목록을 랜덤하게 섞습니다.
random.seed(42)  # 재현 가능한 결과를 위해 시드 설정
random.shuffle(csv_files)

In [ ]:
# 첫 번째 파일에서 컬럼 이름을 가져옵니다.
first_file_path = os.path.join(directory, csv_files[0])
first_df = pd.read_csv(first_file_path)

In [ ]:
separator_token = 999
separator = pd.DataFrame({col: separator_token for col in first_df.columns}, index=[0])

In [8]:
df_list = []
for file in csv_files:
    file_path = os.path.join(directory, file)
    df = pd.read_csv(file_path)
    df = df.iloc[300:-100]
    df_list.append(df)
    df_list.append(separator) 

KeyboardInterrupt: 

In [ ]:
combined_df = pd.concat(df_list, ignore_index=True)

In [ ]:
combined_df

In [ ]:
len(combined_df)

In [ ]:
# Rotation 컬럼만 선택
rotation_columns = [col for col in combined_df.columns if 'Rot' in col]

# Position 컬럼만 선택
position_columns = [col for col in combined_df.columns if 'Pos' in col]

# DataFrame 분리
rotation_df = combined_df[rotation_columns]
position_df = combined_df[position_columns]

# Rotation 컬럼만 -180~180 사이로 정규화
normalize_angle = lambda x:x if x == 999 else (x - 360) if x > 180 else (x + 360) if x < -180 else x
rotation_df = rotation_df.apply(lambda col: col.apply(normalize_angle))

# Position 컬럼만 0~1 사이로 정규화
def normalize_columns(df):
    for col in df.columns:
        if 'Pos' in col:  # 위치에 대한 컬럼만 정규화
            # 999 값을 제외하고 min 및 max를 계산
            filtered_df = df[df[col] != 999]
            min_val = filtered_df[col].min()
            max_val = filtered_df[col].max()

            # 999가 아닌 값만 정규화
            df.loc[df[col] != 999, col] = (df[col] - min_val) / (max_val - min_val)
    return df

position_df = normalize_columns(position_df)

In [ ]:
# -180 ~ 180 범위를 벗어나는 값이 있는지 확인
num_values_out_of_range = rotation_df.apply(lambda col: col.apply(lambda x: x != 999 and (x > 180 or x < -180))).sum().sum()

# 결과 확인
if num_values_out_of_range > 0:
    print(f"범위를 벗어나는 값의 수: {num_values_out_of_range}")
else:
    print("범위를 벗어나는 값이 없습니다.")

In [ ]:
# 조인트 이름 추출
joint_names = [col.split('PosX')[0] for col in position_df.columns if 'PosX' in col]

# 빈 데이터 프레임 생성
posrot_df = pd.DataFrame()

# 각 조인트에 대해 위치 데이터와 회전 데이터를 순차적으로 배열
for joint in joint_names:
    posrot_df[f'{joint}PosX'] = position_df[f'{joint}PosX']
    posrot_df[f'{joint}PosY'] = position_df[f'{joint}PosY']
    posrot_df[f'{joint}PosZ'] = position_df[f'{joint}PosZ']
    posrot_df[f'{joint}RotX'] = rotation_df[f'{joint}RotX']
    posrot_df[f'{joint}RotY'] = rotation_df[f'{joint}RotY']
    posrot_df[f'{joint}RotZ'] = rotation_df[f'{joint}RotZ']

In [ ]:
# Position 컬럼은 그대로 두고, 다시 합치기
posrot_df

# Transformer Learning

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math
from torch_geometric.nn import GCNConv
from torch.utils.data import DataLoader, Dataset, random_split
from torchsummary import summary
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
# 변환된 데이터를 DataFrame으로 변환
posrot_df = pd.DataFrame(posrot_df, columns=posrot_df.columns)

In [ ]:
posrot_df

In [ ]:
train = posrot_df.iloc[:1000400]
test = posrot_df.iloc[1000400:]

In [ ]:
X_train, X_val = train_test_split(train, test_size=0.2, random_state=42, shuffle=False)

In [ ]:
batch_size = 128
n_input = 30 # 시퀀스 수

In [ ]:
class TimeseriesDataset(Dataset):
    def __init__(self, data, sequence_length):
        self.data = data
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.data) - self.sequence_length + 1

    def __getitem__(self, index):
        x = self.data[index:index+self.sequence_length] # : 입력 시퀀스 (30개의 데이터)
        y = self.data[index+self.sequence_length-1] # : 예측 시퀀스 (1개의 데이터)
        return torch.from_numpy(x).float(), torch.from_numpy(y).float()

# 데이터셋과 데이터 로더를 생성
train_dataset = TimeseriesDataset(X_train.values, n_input)
val_dataset = TimeseriesDataset(X_val.values, n_input)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

In [ ]:
# 데이터 로더에서 첫 번째 배치를 가져와서 형태를 확인
first_batch = next(iter(train_loader))
X, y = first_batch

print("X batch shape:", X.shape)
print("y batch shape:", y.shape)

# GCN skeleton joint learning

In [ ]:
joint_names = ['Pelvis', 'L_Hip', 'L_Knee', 'L_Ankle', 'L_Foot', 
                'R_Hip', 'R_Knee', 'R_Ankle', 'R_Foot', 'Spine1', 'Spine2',
               'L_Collar', 'L_Shoulder', 'L_Elbow', 'L_Wrist', 'Neck', 'Head',
               'R_Collar', 'R_Shoulder', 'R_Elbow', 'R_Wrist']
               

In [ ]:
connections = [
    ('Pelvis', 'L_Hip'), ('Pelvis', 'R_Hip'),
    ('L_Hip', 'L_Knee'), ('R_Hip', 'R_Knee'),
    ('L_Knee', 'L_Ankle'), ('R_Knee', 'R_Ankle'),
    ('L_Ankle', 'L_Foot'), ('R_Ankle', 'R_Foot'),
    ('Spine1', 'Spine2'), ('Spine2', 'Pelvis'),
    ('Spine1', 'Neck'), ('Neck', 'Head'),
    ('Spine1', 'L_Collar'), ('Spine1', 'R_Collar'),
    ('L_Collar', 'L_Shoulder'), ('R_Collar', 'R_Shoulder'),
    ('L_Shoulder', 'L_Elbow'), ('R_Shoulder', 'R_Elbow'),
    ('L_Elbow', 'L_Wrist'), ('R_Elbow', 'R_Wrist')
]

In [ ]:
# 조인트 이름을 인덱스로 매핑
joint_to_index = {joint: idx for idx, joint in enumerate(joint_names)}

# 연결 정보를 인덱스 쌍으로 변환
edge_pairs = [(joint_to_index[joint_from], joint_to_index[joint_to]) for joint_from, joint_to in connections]

# edge_index 생성
edge_index = torch.tensor(edge_pairs, dtype=torch.long).t().contiguous()

In [ ]:
edge_index

In [ ]:
# 사용 가능한 GPU 목록을 출력
available_gpus = torch.cuda.device_count()
print("Available GPUs:", available_gpus)

# 현재 장치를 출력 (GPU 사용 가능시 CUDA 장치, 그렇지 않으면 CPU)
current_device = torch.cuda.current_device() if torch.cuda.is_available() else 'CPU'
print("Current device:", torch.cuda.get_device_name(current_device) if torch.cuda.is_available() else current_device)

In [ ]:
# 하이퍼파라미터 설정
batch_size = 128
num_nodes = 21  # 노드 수 (신체 관절 수)
num_node_features = 6  # 각 노드의 특성 수 (위치 및 회전 정보)
num_classes = 126  # 출력 클래스 수 (전체 특성 수)
sequence_length = 30  # 시퀀스 길이
in_channels = 30
out_channels = 256

lr = 0.0001
patience = 10
min_delta = 0.001
epochs = 100

In [ ]:
class TemporalLayer(nn.Module):
    """ 시간적 특성을 처리하기 위한 레이어 """
    def __init__(self, in_channels, out_channels):
        super(TemporalLayer, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=3, padding=1)
        print("TemporalLayer Initialized:")
        print(self)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # [batch_size, num_nodes * num_node_features, sequence_length]
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        return x

class GCNWithTemporal(nn.Module):
    """ 공간적 및 시간적 특성을 모두 고려하는 GCN 모델 """
    def __init__(self, num_node_features, num_classes, sequence_length, num_nodes):
        super(GCNWithTemporal, self).__init__()
        self.conv1 = GCNConv(num_node_features, 64)
        self.conv2 = GCNConv(64, 128)
        self.conv3 = GCNConv(128, 256)
        self.temporal_layer = TemporalLayer(num_nodes * 256, 256)
        self.out = nn.Linear(7680, num_classes) # [batch_size, 256 * 30] = [batch_size, 7680]
        print("GCNWithTemporal Initialized:")
        print(self)

    def forward(self, x, edge_index):
        # 입력 데이터의 형태 확인 (4차원으로 처리)
        # print("Original input shape to GCNWithTemporal:", x.shape) [128, 30, 21, 6])
        
        # x의 형태를 [batch_size, sequence_length, num_nodes, num_node_features]에서
        # [batch_size * sequence_length, num_nodes, num_node_features]로 변환
        batch_size, sequence_length, num_nodes, num_node_features = x.shape
        x = x.reshape(batch_size * sequence_length, num_nodes, num_node_features)

        # GCN 적용
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.conv2(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.conv3(x, edge_index))
        # print("Shape after GCN layers in GCNWithTemporal:", x.shape) [3840, 21, 256])

        # 원래 형태로 복원 (배치 사이즈와 시퀀스 길이를 유지하며 차원 재구성)
        x = x.view(batch_size, sequence_length, -1)
        # print("Shape before TemporalLayer in GCNWithTemporal:", x.shape) [128, 30, 5376])
       
        # 시간 차원을 처리합니다.
        x = self.temporal_layer(x)
        # print("Shape after TemporalLayer in GCNWithTemporal:", x.shape) [128, 256, 30])

        # 최종 출력 차원을 확인
        x = x.reshape(x.size(0), -1)
        # print("Shape before final linear layer in GCNWithTemporal:", x.shape) [128, 7680])

        x = self.out(x)
        return x


In [ ]:
# 모델, 손실 함수, 최적화 알고리즘 초기화
gcn_model = GCNWithTemporal(num_node_features, num_classes, sequence_length, num_nodes)
optimizer = optim.Adam(gcn_model.parameters(), lr=0.0001)
loss_function = torch.nn.MSELoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5)

In [ ]:
# 모델을 현재 장치로 이동합니다.
gcn_model.to(current_device)

In [ ]:
# 손실을 저장할 리스트 초기화
train_losses = []
val_losses = []

# 가장 좋은 손실 값 초기화
best_loss = float('inf')
early_stopping_counter = 0


In [ ]:
# 학습 루프 함수
def train_loop(gcn_model, dataloader, optimizer, loss_function, device, edge_index, num_nodes, num_node_features, sequence_length):
    gcn_model.train()
    total_loss = 0.0
    progress_bar = tqdm(dataloader, desc="Training", leave=False)
    for data, target in progress_bar:
        # 데이터 형태와 edge_index 검증을 위한 출력
        # print("Original data shape:", data.shape)
        # print("Edge index shape:", edge_index.shape)
        # print("Edge index:", edge_index)

        # 데이터 차원 변경
        edge_index = edge_index.to(device)
        data, target = data.to(device), target.to(device)
        data = data.view(-1, num_nodes, num_node_features, sequence_length)  
        # print("View applied data shape:", data.shape)
        data = data.permute(0, 3, 1, 2)  
        # print("Permute applied data shape:", data.shape)
        target = target.view(-1, num_classes)

        # 데이터 형태 출력
        # print("Transformed data shape:", data.shape)
        
        optimizer.zero_grad()
        output = gcn_model(data, edge_index)
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * data.size(0)
        progress_bar.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(dataloader.dataset)
    return avg_loss

# 검증 루프 함수
def val_loop(gcn_model, dataloader, loss_function, device, edge_index, num_nodes, num_node_features, sequence_length):
    gcn_model.eval()
    total_loss = 0.0
    progress_bar = tqdm(dataloader, desc="Validation", leave=False)
    with torch.no_grad():
        for data, target in progress_bar:
            edge_index = edge_index.to(device)
            data, target = data.to(device), target.to(device)
            data = data.view(-1, num_nodes, num_node_features, sequence_length)  
            data = data.permute(0, 3, 1, 2)  
            target = target.view(-1, num_classes)

            output = gcn_model(data, edge_index)
            loss = loss_function(output, target)

            total_loss += loss.item() * data.size(0)
            progress_bar.set_postfix(loss=loss.item())

        avg_loss = total_loss / len(dataloader.dataset)
        return avg_loss

In [ ]:
# Early Stopping 준비
best_loss = float('inf')
early_stopping_counter = 0

# 메인 학습 루프
for epoch in range(epochs):
    print(f'Epoch {epoch+1}/{epochs}')
    # train_loop 함수 호출 시 sequence_length 추가
    avg_train_loss = train_loop(gcn_model, train_loader, optimizer, loss_function, current_device, edge_index, num_nodes, num_node_features, sequence_length)
    # val_loop 함수 호출 시 sequence_length 추가
    avg_val_loss = val_loop(gcn_model, val_loader, loss_function, current_device, edge_index, num_nodes, num_node_features, sequence_length)

    print(f'Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}')

    scheduler.step(avg_val_loss)

    if avg_val_loss < best_loss - min_delta:
        best_loss = avg_val_loss
        best_model_wts = copy.deepcopy(gcn_model.state_dict())
        torch.save(gcn_model.state_dict(), 'best_model.pth')
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1
        if early_stopping_counter >= patience:
            print("Early stopping")
            break

# 최적 모델 가중치 로드
gcn_model.load_state_dict(torch.load('best_model.pth'))

### Transformer Model

In [ ]:
batch_size = 128
n_input = 30  # Sequence length
n_features = 126  # Number of features
output_units = (21 * 6)  # Output shape
head_size = 256  # Size of attention head
num_heads = 7  # Number of attention heads
ff_dim = 512  # Hidden layer size in feed forward network inside transformer
num_blocks = 4  # Number of transformer blocks
mlp_units = [512, 256, 128]  # Size of the dense layers of the final classifier
dropout_rate = 0.3
pad_value = 0  # 패딩에 사용할 값

In [ ]:
# GCN 모델의 출력을 저장할 리스트 초기화
gcn_outputs = []
targets_list = []

for data, target in train_loader:
    data = data.to(current_device)
    target = target.to(current_device)
    edge_index = edge_index.to(current_device)

    batch_size, seq_len, total_features = data.shape
    if total_features == num_nodes * num_node_features:
        data = data.view(batch_size, seq_len, num_node_features, num_nodes)
        data = data.permute(0, 1, 3, 2)

        try:
            gcn_output = gcn_model(data, edge_index)
            gcn_outputs.append(gcn_output.detach())
            targets_list.append(target)  # 해당 배치의 타겟 데이터 저장
        except RuntimeError as e:
            print(f"An error occurred: {e}")
            break

# GCN 모델의 출력 텐서들의 차원을 확인
for i, output in enumerate(gcn_outputs):
    print(f"Output {i} shape: {output.shape}")

# 모든 배치에 대한 GCN 모델의 출력을 하나의 텐서로 결합
gcn_outputs_tensor = torch.cat(gcn_outputs, dim=0)
print("Combined GCN outputs tensor shape:", gcn_outputs_tensor.shape)

# 시퀀스 분할 및 패딩 추가
transformer_inputs = []
for i in range(0, gcn_outputs_tensor.size(0), n_input):
    seq = gcn_outputs_tensor[i:i+n_input]  # n_input 길이의 시퀀스 추출
    if seq.size(0) < n_input:
        # 패딩이 필요한 경우
        padding = torch.full((n_input - seq.size(0), n_features), pad_value, device=seq.device)
        seq = torch.cat([seq, padding], dim=0)  # 시퀀스 끝에 패딩 추가
    transformer_inputs.append(seq.unsqueeze(0))  # 배치 차원 추가

# transformer_inputs 리스트를 하나의 텐서로 결합
transformer_inputs_tensor = torch.cat(transformer_inputs, dim=0)
print("Transformer inputs tensor shape:", transformer_inputs_tensor.shape)

In [ ]:
targets_tensor = torch.cat(targets_list, dim=0)

# Custom Dataset class to handle inputs and targets together
class CustomDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]
    
# 데이터셋 인스턴스 생성
dataset = CustomDataset(transformer_inputs_tensor, targets_tensor)

# 데이터셋을 훈련 세트와 검증 세트로 분할
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Positional Encoding 정의
class PositionalEncoding(nn.Module):
    def __init__(self, n_features, n_input):
        super(PositionalEncoding, self).__init__() # 상속받은 nn.Module 클래스의 __init__() 메서드 호출
        pe = torch.zeros(n_input, n_features) # 가장 큰 시퀀스 길이인 max_len을 기준으로 모두 0으로 채워진 크기가 (max_len, n_features)인 텐서 생성
        position = torch.arange(0, n_input, dtype=torch.float).unsqueeze(1) # position 텐서 생성

        # div_term을 계산하는 방식 수정
        div_term = torch.exp(torch.arange(0, n_features, 2).float() * (-math.log(10000.0) / n_features)) # div_term 계산
        
        # div_term의 길이를 n_features의 절반으로 조정
        div_term = div_term.repeat_interleave(2)[:n_features] # div_term 텐서 생성

        pe[:, 0::2] = torch.sin(position * div_term[0::2]) # 짝수 인덱스에는 sin 함수 적용
        pe[:, 1::2] = torch.cos(position * div_term[1::2]) # 홀수 인덱스에는 cos 함수 적용
        pe = pe.unsqueeze(0) # pe = [bs, seq_len, n_feautres]
        self.register_buffer('pe', pe) # pe 텐서를 모델의 버퍼로 등록

    def forward(self, x):
        pe = self.pe[:, :x.size(1), :].to(x.device)
        # print(f"x.size(): {x.size()}, pe.size(): {pe.size()}")
        x = x + pe
        return x

# Transformer Block 정의
class TransformerBlock(nn.Module):
    def __init__(self, n_features, num_heads, ff_dim, dropout):
        super(TransformerBlock, self).__init__()
        self.attention = nn.MultiheadAttention(n_features, num_heads, dropout=dropout)
        self.ffn = nn.Sequential(
            nn.Linear(n_features, ff_dim),
            nn.ReLU(),
            nn.Linear(ff_dim, n_features)
        )
        self.norm1 = nn.LayerNorm(n_features)
        self.norm2 = nn.LayerNorm(n_features)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        attention_output, _ = self.attention(x, x, x)
        x = self.norm1(x + self.dropout(attention_output))
        ffn_output = self.ffn(x)
        x = self.norm2(x + self.dropout(ffn_output))
        return x
    
# 모델 정의
class TransformerModel(nn.Module):
    def __init__(self, n_features, num_heads, ff_dim, num_blocks, mlp_units, dropout, n_input):
        super(TransformerModel, self).__init__()
        self.pos_encoder = PositionalEncoding(n_features, n_input)
        self.transformer_blocks = nn.ModuleList([TransformerBlock(n_features, num_heads, ff_dim, dropout) for _ in range(num_blocks)])
        
        self.layers = nn.Sequential()
        # 첫 번째 nn.Linear 층의 입력 차원을 n_features로 설정
        self.layers.add_module("dense_0", nn.Linear(n_features, mlp_units[0]))
        self.layers.add_module("relu_0", nn.ReLU())
        self.layers.add_module("dropout_0", nn.Dropout(dropout))
        self.layers.add_module("norm_0", nn.LayerNorm(mlp_units[0]))

        # 이후 층들에 대한 설정
        for i in range(1, len(mlp_units)):
            self.layers.add_module(f"dense_{i}", nn.Linear(mlp_units[i-1], mlp_units[i]))
            self.layers.add_module(f"relu_{i}", nn.ReLU())
            self.layers.add_module(f"dropout_{i}", nn.Dropout(dropout))
            self.layers.add_module(f"norm_{i}", nn.LayerNorm(mlp_units[i]))

        # 최종 출력 층
        self.out = nn.Linear(mlp_units[-1], n_features)

    def forward(self, x):
        x = self.pos_encoder(x)
        for block in self.transformer_blocks:
            x = block(x)
        x = torch.mean(x, dim=1)
        x = self.layers(x)
        return self.out(x)

transformer_model = TransformerModel(n_features, num_heads, ff_dim, num_blocks, mlp_units, dropout_rate, n_input)


In [ ]:
# Positional Encoding 적용
pos_encoder = PositionalEncoding(n_features=n_features, n_input=n_input)
# transformer_input_sequences = pos_encoder(transformer_input_sequences)

In [ ]:
column_order = [
    'm_avg_PelvisPosX', 'm_avg_PelvisPosY', 'm_avg_PelvisPosZ',
    'm_avg_PelvisRotX', 'm_avg_PelvisRotY', 'm_avg_PelvisRotZ',

    'm_avg_L_HipPosX', 'm_avg_L_HipPosY', 'm_avg_L_HipPosZ',
    'm_avg_L_HipRotX', 'm_avg_L_HipRotY', 'm_avg_L_HipRotZ',

    'm_avg_L_KneePosX', 'm_avg_L_KneePosY', 'm_avg_L_KneePosZ',
    'm_avg_L_KneeRotX', 'm_avg_L_KneeRotY', 'm_avg_L_KneeRotZ',

    'm_avg_L_AnklePosX', 'm_avg_L_AnklePosY', 'm_avg_L_AnklePosZ',
    'm_avg_L_AnkleRotX', 'm_avg_L_AnkleRotY', 'm_avg_L_AnkleRotZ',

    'm_avg_L_FootPosX', 'm_avg_L_FootPosY', 'm_avg_L_FootPosZ',
    'm_avg_L_FootRotX', 'm_avg_L_FootRotY', 'm_avg_L_FootRotZ',

    'm_avg_R_HipPosX', 'm_avg_R_HipPosY', 'm_avg_R_HipPosZ',
    'm_avg_R_HipRotX', 'm_avg_R_HipRotY', 'm_avg_R_HipRotZ',

    'm_avg_R_KneePosX', 'm_avg_R_KneePosY', 'm_avg_R_KneePosZ',
    'm_avg_R_KneeRotX', 'm_avg_R_KneeRotY', 'm_avg_R_KneeRotZ',

    'm_avg_R_AnklePosX', 'm_avg_R_AnklePosY', 'm_avg_R_AnklePosZ',
    'm_avg_R_AnkleRotX', 'm_avg_R_AnkleRotY', 'm_avg_R_AnkleRotZ',

    'm_avg_R_FootPosX', 'm_avg_R_FootPosY', 'm_avg_R_FootPosZ',
    'm_avg_R_FootRotX', 'm_avg_R_FootRotY', 'm_avg_R_FootRotZ',

    'm_avg_Spine1PosX', 'm_avg_Spine1PosY', 'm_avg_Spine1PosZ',
    'm_avg_Spine1RotX', 'm_avg_Spine1RotY', 'm_avg_Spine1RotZ',

    'm_avg_Spine2PosX', 'm_avg_Spine2PosY', 'm_avg_Spine2PosZ',
    'm_avg_Spine2RotX', 'm_avg_Spine2RotY', 'm_avg_Spine2RotZ',

    'm_avg_L_CollarPosX', 'm_avg_L_CollarPosY', 'm_avg_L_CollarPosZ',
    'm_avg_L_CollarRotX', 'm_avg_L_CollarRotY', 'm_avg_L_CollarRotZ',

    'm_avg_L_ShoulderPosX', 'm_avg_L_ShoulderPosY', 'm_avg_L_ShoulderPosZ',
    'm_avg_L_ShoulderRotX', 'm_avg_L_ShoulderRotY', 'm_avg_L_ShoulderRotZ',

    'm_avg_L_ElbowPosX', 'm_avg_L_ElbowPosY', 'm_avg_L_ElbowPosZ',
    'm_avg_L_ElbowRotX', 'm_avg_L_ElbowRotY', 'm_avg_L_ElbowRotZ',

    'm_avg_L_WristPosX', 'm_avg_L_WristPosY', 'm_avg_L_WristPosZ',
    'm_avg_L_WristRotX', 'm_avg_L_WristRotY', 'm_avg_L_WristRotZ',

    'm_avg_NeckPosX', 'm_avg_NeckPosY', 'm_avg_NeckPosZ',
    'm_avg_NeckRotX', 'm_avg_NeckRotY', 'm_avg_NeckRotZ',

    'm_avg_HeadPosX', 'm_avg_HeadPosY', 'm_avg_HeadPosZ',
    'm_avg_HeadRotX', 'm_avg_HeadRotY', 'm_avg_HeadRotZ',

    'm_avg_R_CollarPosX', 'm_avg_R_CollarPosY', 'm_avg_R_CollarPosZ',
    'm_avg_R_CollarRotX', 'm_avg_R_CollarRotY', 'm_avg_R_CollarRotZ',

    'm_avg_R_ShoulderPosX', 'm_avg_R_ShoulderPosY', 'm_avg_R_ShoulderPosZ',
    'm_avg_R_ShoulderRotX', 'm_avg_R_ShoulderRotY', 'm_avg_R_ShoulderRotZ',

    'm_avg_R_ElbowPosX', 'm_avg_R_ElbowPosY', 'm_avg_R_ElbowPosZ',
    'm_avg_R_ElbowRotX', 'm_avg_R_ElbowRotY', 'm_avg_R_ElbowRotZ',
    
    'm_avg_R_WristPosX', 'm_avg_R_WristPosY', 'm_avg_R_WristPosZ',
    'm_avg_R_WristRotX', 'm_avg_R_WristRotY', 'm_avg_R_WristRotZ'
    ]

In [ ]:
column_names = [
    'm_avg_HeadPosX', 'm_avg_HeadPosY', 'm_avg_HeadPosZ',
    'm_avg_L_WristPosX', 'm_avg_L_WristPosY', 'm_avg_L_WristPosZ',
    'm_avg_R_WristPosX', 'm_avg_R_WristPosY', 'm_avg_R_WristPosZ',
    'm_avg_L_FootPosX', 'm_avg_L_FootPosY', 'm_avg_L_FootPosZ',
    'm_avg_R_FootPosX', 'm_avg_R_FootPosY', 'm_avg_R_FootPosZ',
    'm_avg_HeadRotX', 'm_avg_HeadRotY', 'm_avg_HeadRotZ',
    'm_avg_L_WristRotX', 'm_avg_L_WristRotY', 'm_avg_L_WristRotZ',
    'm_avg_R_WristRotX', 'm_avg_R_WristRotY', 'm_avg_R_WristRotZ',
    'm_avg_L_FootRotX', 'm_avg_L_FootRotY', 'm_avg_L_FootRotZ',
    'm_avg_R_FootRotX', 'm_avg_R_FootRotY', 'm_avg_R_FootRotZ',
]

weighted_columns_indices = [column_order.index(name) for name in column_names]
print(weighted_columns_indices)

In [ ]:
class CustomLoss(nn.Module):
    def __init__(self, weighted_columns_indices, weight_for_weighted_columns, threshold, penalty_weight):
        super(CustomLoss, self).__init__()
        self.weighted_columns_indices = torch.tensor(weighted_columns_indices)
        self.weight_for_weighted_columns = weight_for_weighted_columns
        self.threshold = threshold  # 신체 움직임의 임계값
        self.penalty_weight = penalty_weight  # 비정상적 움직임에 대한 패널티 가중치

    def forward(self, y_true, y_pred):
        # 각 조인트의 6개 특성에 대한 손실을 계산하기 위해 y_true와 y_pred의 마지막 차원을 제거
        y_pred_values = y_pred[:, :]

        # MSE 계산
        mse = F.mse_loss(y_true[:, :], y_pred_values, reduction='none')
        mse = mse.mean(axis=-1)

        # 특정 joint에 대한 가중치 적용
        weighted_mse = y_pred[:, self.weighted_columns_indices]
        weighted_mse = (weighted_mse ** 2) * self.weight_for_weighted_columns
        mse += weighted_mse.mean(axis=-1)

        # 과도한 움직임에 대한 패널티 적용
        excessive_movement_penalty = (y_pred - y_true).abs() > self.threshold
        penalty = excessive_movement_penalty.type(torch.float32) * self.penalty_weight
        mse += penalty.mean(axis=-1)

        return mse.mean()  # 전체 배치에 대한 평균 손실 반환

# 가중치를 적용할 열 인덱스와 가중치 값
weighted_columns_indices = weighted_columns_indices
weight_for_weighted_columns = 2.0

# 임계값과 패널티 가중치 설정
threshold = 10.0  # 예시 임계값
penalty_weight = 0.8  # 예시 패널티 가중치

# CustomLoss 인스턴스 생성
custom_loss_instance = CustomLoss(
    weighted_columns_indices=weighted_columns_indices,
    weight_for_weighted_columns=weight_for_weighted_columns,
    threshold=threshold,
    penalty_weight=penalty_weight
)

In [ ]:
# 최적화기와 손실 함수
optimizer = optim.Adam(transformer_model.parameters(), lr=0.0001)
criterion = custom_loss_instance

In [ ]:
# 설정
epochs = 500
patience = 7  # Early Stopping patience
best_loss = np.inf
early_stopping_counter = 0

In [ ]:
# Learning Rate Scheduler 설정
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.2, patience=7, min_lr=0.00001)

In [ ]:
# 사용 가능한 GPU 목록을 출력
available_gpus = torch.cuda.device_count()
print("Available GPUs:", available_gpus)

# 현재 장치를 출력 (GPU 사용 가능시 CUDA 장치, 그렇지 않으면 CPU)
current_device = torch.cuda.current_device() if torch.cuda.is_available() else 'CPU'
print("Current device:", torch.cuda.get_device_name(current_device) if torch.cuda.is_available() else current_device)

In [ ]:
transformer_model = transformer_model.to(current_device)

In [ ]:
summary(transformer_model, input_size=(n_input, n_features))

In [ ]:
for epoch in range(epochs):
    transformer_model.train()
    total_train_loss = 0.0

    train_progress_bar = tqdm(train_loader, desc=f'Train Epoch {epoch+1}/{epochs}', leave=False)
    for batch_inputs, batch_targets in train_progress_bar:
        batch_inputs = batch_inputs.to(current_device)
        batch_targets = batch_targets.to(current_device)

        optimizer.zero_grad()
        outputs = transformer_model(batch_inputs)
        loss = criterion(outputs, batch_targets)
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item() * batch_inputs.size(0)

    avg_train_loss = total_train_loss / len(train_dataset)
    train_losses.append(avg_train_loss)

    # Validation phase
    transformer_model.eval()
    total_val_loss = 0.0
    val_progress_bar = tqdm(val_loader, desc=f'Validation Epoch {epoch+1}/{epochs}', leave=False)
    with torch.no_grad():
        for batch_inputs, batch_targets in val_progress_bar:
            batch_inputs = batch_inputs.to(current_device)
            batch_targets = batch_targets.to(current_device)

            outputs = transformer_model(batch_inputs)
            loss = criterion(outputs, batch_targets)
            total_val_loss += loss.item() * batch_inputs.size(0)

    avg_val_loss = total_val_loss / len(val_dataset)
    val_losses.append(avg_val_loss)

    print(f'Epoch {epoch+1}/{epochs} - Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}')

    scheduler.step(avg_val_loss)

    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        best_model_wts = copy.deepcopy(transformer_model.state_dict())
        torch.save({'transformer_model': transformer_model.state_dict(), 'gcn_model': gcn_model.state_dict()}, 'best_model.pth')
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1
        if early_stopping_counter >= patience:
            print("Early stopping triggered")
            break

# 최고의 모델 가중치 로드
transformer_model.load_state_dict(best_model_wts)


In [ ]:
torch.save(transformer_model.state_dict(), 'final_model_Transformer.pth')

In [ ]:
plt.plot(train_losses, label='Training Loss', color='blue')
plt.plot(val_losses, label='Validation Loss', color='orange')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Test with Real Data

In [ ]:
test_df = pd.read_csv('./B01_TransformData_FinalAvatar_20230922_171230.csv').iloc[300:-100]

In [ ]:
test_df.head()

In [ ]:
test_rotation_columns = [col for col in test_df.columns if 'Rot' in col]

test_position_columns = [col for col in test_df.columns if 'Pos' in col]

# DataFrame 분리
test_rotation_df = test_df[test_rotation_columns]
test_position_df = test_df[test_position_columns]

# Rotation 컬럼만 -180~180 사이로 정규화
normalize_angle = lambda x:x if x == 999 else (x - 360) if x > 180 else (x + 360) if x < -180 else x
test_rotation_df = test_rotation_df.apply(lambda col: col.apply(normalize_angle))

# Position 컬럼만 0~1 사이로 정규화
def normalize_columns(df):
    for col in df.columns:
        if 'Pos' in col:  # 위치에 대한 컬럼만 정규화
            min_val = df[col].min()
            max_val = df[col].max()
            df[col] = (df[col] - min_val) / (max_val - min_val)
    return df

test_position_df = normalize_columns(test_position_df)

In [ ]:
# -180 ~ 180 범위를 벗어나는 값이 있는지 확인
num_values_out_of_range = test_rotation_df.apply(lambda col: col.apply(lambda x: x != 999 and (x > 180 or x < -180))).sum().sum()

# 결과 확인
if num_values_out_of_range > 0:
    print(f"범위를 벗어나는 값의 수: {num_values_out_of_range}")
else:
    print("범위를 벗어나는 값이 없습니다.")

In [ ]:
# 조인트 이름 추출
joint_names = [col.split('PosX')[0] for col in test_position_df.columns if 'PosX' in col]

# 빈 데이터 프레임 생성
test_posrot_df = pd.DataFrame()

# 각 조인트에 대해 위치 데이터와 회전 데이터를 순차적으로 배열
for joint in joint_names:
    test_posrot_df[f'{joint}PosX'] = test_position_df[f'{joint}PosX']
    test_posrot_df[f'{joint}PosY'] = test_position_df[f'{joint}PosY']
    test_posrot_df[f'{joint}PosZ'] = test_position_df[f'{joint}PosZ']
    test_posrot_df[f'{joint}RotX'] = test_rotation_df[f'{joint}RotX']
    test_posrot_df[f'{joint}RotY'] = test_rotation_df[f'{joint}RotY']
    test_posrot_df[f'{joint}RotZ'] = test_rotation_df[f'{joint}RotZ']

In [ ]:
test_posrot_df

In [ ]:
# 결과를 새로운 CSV 파일로 저장합니다.
test_posrot_df.to_csv('./test_posrot_df.csv', index=True)

In [ ]:
# 모델 불러오기
model = TransformerModel(
    n_features=n_features,
    num_heads=num_heads,
    ff_dim=ff_dim,
    num_blocks=num_blocks,
    mlp_units=mlp_units,
    dropout=dropout_rate,
    n_input=n_input
)

model.load_state_dict(torch.load('final_model_Transformer.pth'))

# 모델을 평가 모드로 설정
model.eval()

# 예측 값을 넣을 빈 리스트
test_predictions = []

# 훈련 데이터셋에서 마지막 입력 개수의 값을 가져온 후
current_batch = torch.from_numpy(test_posrot_df[-n_input:].values.astype(np.float32)).reshape((1, n_input, n_features))

# 모델이 사용하는 디바이스를 확인하고 데이터를 해당 디바이스로 옮깁니다.
current_device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(current_device)
current_batch = current_batch.to(current_device)

# 예측 과정 반복
with torch.no_grad():  # 그래디언트 계산을 비활성화
    for i in range(1):
        # 현재 배치에서 다음 포인트를 예측
        current_pred = model(current_batch).cpu().numpy()[0]  # 마지막 시퀀스 포인트 예측

        # 예측된 마지막 프레임을 리스트에 추가
        test_predictions.append(current_pred)

        # 새로운 배치 생성: 마지막 시퀀스 제외하고 예측값 추가
        current_batch = np.roll(current_batch.cpu().numpy(), -1, axis=1)
        current_batch[:, -1, :] = current_pred
        current_batch = torch.from_numpy(current_batch).to(current_device)

In [ ]:
test_predictions_array = np.array(test_predictions)

In [ ]:
# 변환된 배열을 데이터프레임으로 변환합니다. 이때 column_order 리스트를 열 이름으로 사용합니다.
test_predictions = pd.DataFrame(test_predictions_array, columns=column_order)

# test_predictions 데이터프레임을 CSV 파일로 저장합니다.
test_predictions.to_csv('./test_predictions.csv', index=True)


In [ ]:
test_predictions

In [ ]:
# input 데이터(test_df)의 마지막 30 프레임과 
last_inputs_df = test_posrot_df.iloc[-30:][column_order].reset_index(drop=True)
test_predictions_df = pd.DataFrame(test_predictions_array, columns=column_order)

test_combined_df = pd.concat([last_inputs_df, test_predictions_df], ignore_index=True)

In [ ]:
test_combined_df

In [ ]:
test_combined_df.to_csv('./test_combined_df.csv', index=True)

# Visualize

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

In [ ]:
def visualize_skeleton(df, connections):
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')

    # Plot each joint
    for joint in joint_names:
        ax.scatter(df[f'{joint}PosX'], df[f'{joint}PosY'], df[f'{joint}PosZ'], label=joint)

    # Draw lines (bones) connecting the joints
    for connection in connections:
        joint_from, joint_to = connection
        # Update to use the correct column names
        ax.plot(
            [df[f'm_avg_{joint_from}PosX'], df[f'm_avg_{joint_to}PosX']],
            [df[f'm_avg_{joint_from}PosY'], df[f'm_avg_{joint_to}PosY']],
            [df[f'm_avg_{joint_from}PosZ'], df[f'm_avg_{joint_to}PosZ']],
            'k-'
        )

    # Set labels and legend
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.legend(loc='best')

    # Set the initial viewing angle
    ax.view_init(elev=20., azim=-90)

    plt.show()

visualize_skeleton(test_predictions.iloc[0], connections)